In [ ]:
import pandas as pd
import numpy as np
# import pinball loss from sklearn
from sklearn.metrics import mean_pinball_loss

In [ ]:
# Configuration settings
from config.simulation_setting import Simulation

sim_params = Simulation.testing_period  # Simulation parameters

In [ ]:
# load csv file
if sim_params['most_recent']:
    df = pd.read_csv('wp_forecasters_comparison_results.csv')
else:
    df = pd.read_csv('wp_forecasters_comparison_results_no_mostrecent.csv')
print(df.shape)

# min and max values for datetime
min_date = df['datetime'].min()
max_date = df['datetime'].max()
print('Min date:', min_date)
print('Max date:', max_date)

df_clean = df.rename(columns={'10_predictions': 'q10_QRA', '50_predictions': 'q50_QRA', '90_predictions': 'q90_QRA'})
df_clean.head()

In [ ]:
# join the two dataframes
df = df_clean.copy() #.set_index('datetime').join(df_stack_clean.set_index('datetime')[['q10_stack', 'q50_stack', 'q90_stack']], on='datetime').reset_index()
# add date column
df['date'] = pd.to_datetime(df['datetime']).dt.date
# add month-year column
df['month_year'] = pd.to_datetime(df['datetime']).dt.to_period('M')
df.head()

In [ ]:
# calculate rmse
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# calculate mean pinball loss with sklearn
def mean_pinball_loss_sklearn(targets, predictions, tau):
    return mean_pinball_loss(targets, predictions, alpha=tau)


def WIS(y_true, lower, upper, alpha):
    " Winkler Interval Score (WIS) for a single row"

    assert np.isnan(y_true) == False, "y_true contains NaN value(s)"
    assert np.isinf(y_true) == False, "y_true contains inf values(s)"
    assert np.isnan(lower)  == False, "lower interval value contains NaN value(s)"
    assert np.isinf(lower)  == False, "lower interval value contains inf values(s)"
    assert np.isnan(upper)  == False, "upper interval value contains NaN value(s)"
    assert np.isinf(upper)  == False, "upper interval value contains inf values(s)"
    assert alpha > 0 and alpha <= 1,  f"alpha should be (0,1]. Found: {alpha}"

    # WIS for one single row
    score = np.abs(upper-lower)
    if y_true < np.minimum(upper,lower):
        score += ((2/alpha) * (np.minimum(upper, lower) - y_true))
    if y_true > np.maximum(upper,lower):
        score += ((2/alpha) * (y_true - np.maximum(upper,lower)))
    return score

# vectorize the function
v_WIS = np.vectorize(WIS)

def mean_winkler_score(y_true, lower, upper, alpha):
    " Mean Winkler Interval Score (MWIS) for a pandas Series or 1D array"
    assert y_true.ndim == 1, "y_true: pandas Series or 1D array expected"
    assert lower.ndim  == 1, "lower: pandas Series or 1D array expected"
    assert upper.ndim  == 1, "upper: pandas Series or 1D array expected"
    assert isinstance(alpha, float) == True, "alpha: float expected"
    WIS_scores = v_WIS(y_true,lower,upper,alpha)
    MWIS      = np.mean(WIS_scores)
    MWIS      = float(MWIS)
    return MWIS

In [ ]:
def compute_rmse_testing_days(df, predictions, targets = 'measured', date_column = 'date'):
    " Compute RMSE for each day in the dataframe "
    rmse_list = []
    for date in df[date_column].unique():
        date_df = df[df[date_column] == date]
        rmse_list.append(rmse(date_df[targets], date_df[predictions]))
    return rmse_list

def compute_pinball_loss_testing_days(df, tau, predictions, targets = 'measured', date_column = 'date'):
    " Compute Pinball Loss for each day in the dataframe "
    pinball_loss_list = []
    for date in df[date_column].unique():
        date_df = df[df[date_column] == date]
        pinball_loss_list.append(mean_pinball_loss_sklearn(date_df[targets], date_df[predictions], tau))
    return pinball_loss_list

def compute_winkler_score_testing_days(df, alpha, predictions_q10, predictions_q90, targets = 'measured', date_column = 'date'):
    " Compute Winkler Score for each day in the dataframe "
    winkler_score_list = []
    for date in df[date_column].unique():
        date_df = df[df[date_column] == date]
        winkler_score_list.append(mean_winkler_score(date_df[targets], date_df[predictions_q10], date_df[predictions_q90], alpha))
    return winkler_score_list

def compute_coverage_testing_days(df, predictions_q10, predictions_q90, targets = 'measured', date_column = 'date'):
    " Compute coverage for each day in the dataframe "
    coverage_list = []
    for date in df[date_column].unique():
        date_df = df[df[date_column] == date]
        coverage_list.append(np.mean((date_df[predictions_q10] <= date_df[targets]) & (date_df[predictions_q90] >= date_df[targets])))
    return coverage_list

def compute_sharpness_testing_days(df, predictions_q10, predictions_q90, date_column = 'date'):
    " Compute sharpness for each day in the dataframe "
    sharpness_list = []
    for date in df[date_column].unique():
        date_df = df[df[date_column] == date]
        sharpness_list.append(np.mean(date_df[predictions_q90] - date_df[predictions_q10]))
    return sharpness_list

In [ ]:
# plot rmse
import matplotlib.pyplot as plt

def plot_ts_loss(df, list_loss_1, list_loss_2, model_name_1, model_name_2, date_column = 'date'):
    """ Plot timeseries loss"""
    assert len(list_loss_1) == len(list_loss_2), "The two lists must have the same length"
    assert len(list_loss_1) == len(df[date_column].unique()), "The length of the list must be equal to the number of unique dates in the dataframe"
    assert len(list_loss_2) == len(df[date_column].unique()), "The length of the list must be equal to the number of unique dates in the dataframe"
    # plot ensemble
    plt.figure(figsize=(25, 7))
    # x-axis date, y-axis rmse
    if date_column == 'date':
        timestamps = df[date_column].unique()
    else:
        timestamps = df[date_column].unique().to_timestamp()
    plt.plot(timestamps, list_loss_1, label=model_name_1)
    plt.plot(timestamps, list_loss_2, label=model_name_2, alpha=0.5, linestyle='dashed')
    # fill between the two models
    plt.fill_between(timestamps, list_loss_1, list_loss_2, color='red', alpha=0.3)
    plt.title('RMSE by date')
    plt.ylabel('RMSE')
    plt.xlabel('Date')
    plt.legend()
    plt.show()

In [ ]:
# ----- Date-based analysis ----- 

# results for the ensemble model QRA
# calculate rmse
rmse_list_QRA = compute_rmse_testing_days(df, predictions='q50_QRA')
# calculate pinball loss
pinball_loss_01_list_QRA = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_QRA')
pinball_loss_09_list_QRA = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_QRA')
# calculate Winkler Interval Score
winkler_score_list_QRA = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='q10_QRA', predictions_q90='q90_QRA')
# calculate coverage
coverage_list_QRA = compute_coverage_testing_days(df, predictions_q10='q10_QRA', predictions_q90='q90_QRA')
# calculate sharpness
sharpness_list_QRA = compute_sharpness_testing_days(df, predictions_q10='q10_QRA', predictions_q90='q90_QRA')

# results for the day-ahead model
# calculate rmse
rmse_list_dayahead = compute_rmse_testing_days(df, predictions='dayaheadforecast')
# calculate pinball loss
pinball_loss_01_list_dayahead = compute_pinball_loss_testing_days(df, tau=0.1, predictions='dayaheadconfidence10')
pinball_loss_09_list_dayahead = compute_pinball_loss_testing_days(df, tau=0.9, predictions='dayaheadconfidence90')
# calculate winkler score
winkler_score_list_dayahead = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='dayaheadconfidence10', predictions_q90='dayaheadconfidence90')
# calculate coverage
coverage_list_dayahead = compute_coverage_testing_days(df, predictions_q10='dayaheadconfidence10', predictions_q90='dayaheadconfidence90')
# calculate sharpness
sharpness_list_dayahead = compute_sharpness_testing_days(df, predictions_q10='dayaheadconfidence10', predictions_q90='dayaheadconfidence90')

# results for dayahead11hforecast model
# calculate rmse
rmse_list_dayahead11h = compute_rmse_testing_days(df, predictions='dayahead11hforecast')
# calculate pinball loss
pinball_loss_01_list_dayahead11h = compute_pinball_loss_testing_days(df, tau=0.1, predictions='dayahead11hconfidence10')
pinball_loss_09_list_dayahead11h = compute_pinball_loss_testing_days(df, tau=0.9, predictions='dayahead11hconfidence90')
# calculate winkler score
winkler_score_list_dayahead11h = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='dayahead11hconfidence10', predictions_q90='dayahead11hconfidence90')
# calculate coverage
coverage_list_dayahead11h = compute_coverage_testing_days(df, predictions_q10='dayahead11hconfidence10', predictions_q90='dayahead11hconfidence90')
# calculate sharpness
sharpness_list_dayahead11h = compute_sharpness_testing_days(df, predictions_q10='dayahead11hconfidence10', predictions_q90='dayahead11hconfidence90')

# results for weekaheadforecast model
# calculate rmse
rmse_list_weekahead = compute_rmse_testing_days(df, predictions='weekaheadforecast')
# calculate pinball loss
pinball_loss_01_list_weekahead = compute_pinball_loss_testing_days(df, tau=0.1, predictions='weekaheadconfidence10')
pinball_loss_09_list_weekahead = compute_pinball_loss_testing_days(df, tau=0.9, predictions='weekaheadconfidence90')
# calculate winkler score
winkler_score_list_weekahead = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='weekaheadconfidence10', predictions_q90='weekaheadconfidence90')
# calculate coverage
coverage_list_weekahead = compute_coverage_testing_days(df, predictions_q10='weekaheadconfidence10', predictions_q90='weekaheadconfidence90')
# calculate sharpness
sharpness_list_weekahead = compute_sharpness_testing_days(df, predictions_q10='weekaheadconfidence10', predictions_q90='weekaheadconfidence90')

# results for 'q10_best_model', 'q50_best_model', 'q90_best_model' model
# calculate rmse
rmse_list_best = compute_rmse_testing_days(df, predictions='q50_best_model')
# calculate pinball loss
pinball_loss_01_list_best = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_best_model')
pinball_loss_09_list_best = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_best_model')
# calculate winkler score
winkler_score_list_best = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='q10_best_model', predictions_q90='q90_best_model')
# calculate coverage
coverage_list_best = compute_coverage_testing_days(df, predictions_q10='q10_best_model', predictions_q90='q90_best_model')
# calculate sharpness
sharpness_list_best = compute_sharpness_testing_days(df, predictions_q10='q10_best_model', predictions_q90='q90_best_model')

# results for 'q10_weight_avg', 'q50_weight_avg', 'q90_weight_avg' model
# calculate rmse
rmse_list_weight_avg = compute_rmse_testing_days(df, predictions='q50_weight_avg')
# calculate pinball loss
pinball_loss_01_list_weight_avg = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_weight_avg')
pinball_loss_09_list_weight_avg = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_weight_avg')
# calculate winkler score
winkler_score_list_weight_avg = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='q10_weight_avg', predictions_q90='q90_weight_avg')
# calculate coverage
coverage_list_weight_avg = compute_coverage_testing_days(df, predictions_q10='q10_weight_avg', predictions_q90='q90_weight_avg')
# calculate sharpness
sharpness_list_weight_avg = compute_sharpness_testing_days(df, predictions_q10='q10_weight_avg', predictions_q90='q90_weight_avg')

# results for 'q10_weight_avg_soft', 'q50_weight_avg_soft', 'q90_weight_avg_soft' model
# calculate rmse
rmse_list_weight_avg_soft = compute_rmse_testing_days(df, predictions='q50_weight_avg_soft')
# calculate pinball loss
pinball_loss_01_list_weight_avg_soft = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_weight_avg_soft')
pinball_loss_09_list_weight_avg_soft = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_weight_avg_soft')
# calculate winkler score
winkler_score_list_weight_avg_soft = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='q10_weight_avg_soft', predictions_q90='q90_weight_avg_soft')
# calculate coverage
coverage_list_weight_avg_soft = compute_coverage_testing_days(df, predictions_q10='q10_weight_avg_soft', predictions_q90='q90_weight_avg_soft')
# calculate sharpness
sharpness_list_weight_avg_soft = compute_sharpness_testing_days(df, predictions_q10='q10_weight_avg_soft', predictions_q90='q90_weight_avg_soft')

# results for 'q10_equal_weights', 'q50_equal_weights', 'q90_equal_weights' model
# calculate rmse
rmse_list_equal_weights = compute_rmse_testing_days(df, predictions='q50_equal_weights')
# calculate pinball loss
pinball_loss_01_list_equal_weights = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_equal_weights')
pinball_loss_09_list_equal_weights = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_equal_weights')
# calculate winkler score
winkler_score_list_equal_weights = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='q10_equal_weights', predictions_q90='q90_equal_weights')
# calculate coverage
coverage_list_equal_weights = compute_coverage_testing_days(df, predictions_q10='q10_equal_weights', predictions_q90='q90_equal_weights')
# calculate sharpness
sharpness_list_equal_weights = compute_sharpness_testing_days(df, predictions_q10='q10_equal_weights', predictions_q90='q90_equal_weights')

if sim_params['most_recent']:
    # results for 'mostrecentconfidence10', 'mostrecentconfidence90', 'mostrecentforecast' model
    # calculate rmse
    rmse_list_most_recent = compute_rmse_testing_days(df, predictions='mostrecentforecast')
    # calculate pinball loss
    pinball_loss_01_list_most_recent = compute_pinball_loss_testing_days(df, tau=0.1, predictions='mostrecentconfidence10')
    pinball_loss_09_list_most_recent = compute_pinball_loss_testing_days(df, tau=0.9, predictions='mostrecentconfidence90')
    # calculate winkler score
    winkler_score_list_most_recent = compute_winkler_score_testing_days(df, alpha=0.1, predictions_q10='mostrecentconfidence10', predictions_q90='mostrecentconfidence90')
    # calculate coverage
    coverage_list_most_recent = compute_coverage_testing_days(df, predictions_q10='mostrecentconfidence10', predictions_q90='mostrecentconfidence90')
    # calculate sharpness
    sharpness_list_most_recent = compute_sharpness_testing_days(df, predictions_q10='mostrecentconfidence10', predictions_q90='mostrecentconfidence90')

In [ ]:
# ----- Motnhly-Year-based analysis ----- 

# results for the ensemble model QRA
# calculate rmse
rmse_list_QRA_month_year = compute_rmse_testing_days(df, predictions='q50_QRA', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_QRA_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_QRA', date_column='month_year')
pinball_loss_09_list_QRA_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_QRA', date_column='month_year')

# results for the day-ahead model
# calculate rmse
rmse_list_dayahead_month_year = compute_rmse_testing_days(df, predictions='dayaheadforecast', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_dayahead_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='dayaheadconfidence10', date_column='month_year')
pinball_loss_09_list_dayahead_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='dayaheadconfidence90', date_column='month_year')

# results for dayahead11hforecast model
# calculate rmse
rmse_list_dayahead11h_month_year = compute_rmse_testing_days(df, predictions='dayahead11hforecast', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_dayahead11h_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='dayahead11hconfidence10', date_column='month_year')
pinball_loss_09_list_dayahead11h_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='dayahead11hconfidence90', date_column='month_year')

# results for weekaheadforecast model
# calculate rmse
rmse_list_weekahead_month_year = compute_rmse_testing_days(df, predictions='weekaheadforecast', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_weekahead_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='weekaheadconfidence10', date_column='month_year')
pinball_loss_09_list_weekahead_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='weekaheadconfidence90', date_column='month_year')

# results for 'q10_best_model', 'q50_best_model', 'q90_best_model' model
# calculate rmse
rmse_list_best_month_year = compute_rmse_testing_days(df, predictions='q50_best_model', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_best_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_best_model', date_column='month_year')
pinball_loss_09_list_best_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_best_model', date_column='month_year')

# results for 'q10_weight_avg', 'q50_weight_avg', 'q90_weight_avg' model
# calculate rmse
rmse_list_weight_avg_month_year = compute_rmse_testing_days(df, predictions='q50_weight_avg', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_weight_avg_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_weight_avg', date_column='month_year')
pinball_loss_09_list_weight_avg_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_weight_avg', date_column='month_year')

# results for 'q10_weight_avg_soft', 'q50_weight_avg_soft', 'q90_weight_avg_soft' model
# calculate rmse
rmse_list_weight_avg_soft_month_year = compute_rmse_testing_days(df, predictions='q50_weight_avg_soft', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_weight_avg_soft_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_weight_avg_soft', date_column='month_year')
pinball_loss_09_list_weight_avg_soft_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_weight_avg_soft', date_column='month_year')

# results for 'q10_equal_weights', 'q50_equal_weights', 'q90_equal_weights' model
# calculate rmse
rmse_list_equal_weights_month_year = compute_rmse_testing_days(df, predictions='q50_equal_weights', date_column='month_year')
# calculate pinball loss
pinball_loss_01_list_equal_weights_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='q10_equal_weights', date_column='month_year')
pinball_loss_09_list_equal_weights_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='q90_equal_weights', date_column='month_year')

if sim_params['most_recent']:
    # results for 'mostrecentconfidence10', 'mostrecentconfidence90', 'mostrecentforecast' model
    # calculate rmse
    rmse_list_most_recent_month_year = compute_rmse_testing_days(df, predictions='mostrecentforecast', date_column='month_year')
    # calculate pinball loss
    pinball_loss_01_list_most_recent_month_year = compute_pinball_loss_testing_days(df, tau=0.1, predictions='mostrecentconfidence10', date_column='month_year')
    pinball_loss_09_list_most_recent_month_year = compute_pinball_loss_testing_days(df, tau=0.9, predictions='mostrecentconfidence90', date_column='month_year')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def barplot(sim_params, title, loss_list_QRA, loss_list_dayahead, loss_list_dayahead11h, loss_list_weekahead, loss_list_best, loss_list_weight_avg, loss_list_weight_avg_soft, loss_list_equal_weights, loss_list_most_recent=None):
        " Plot barplot of mean rmse with std"
        # compute mean rmse and std
        mean_loss_QRA, err_loss_QRA = np.mean(loss_list_QRA), np.std(loss_list_QRA)/np.sqrt(len(loss_list_QRA))
        mean_loss_dayahead, err_loss_dayahead = np.mean(loss_list_dayahead), np.std(loss_list_dayahead)/np.sqrt(len(loss_list_dayahead))
        mean_loss_dayahead11h, err_loss_dayahead11h = np.mean(loss_list_dayahead11h), np.std(loss_list_dayahead11h)/np.sqrt(len(loss_list_dayahead11h))
        mean_loss_weekahead, err_loss_weekahead = np.mean(loss_list_weekahead), np.std(loss_list_weekahead)/np.sqrt(len(loss_list_weekahead))
        mean_loss_best_model, err_loss_best_model = np.mean(loss_list_best), np.std(loss_list_best)/np.sqrt(len(loss_list_best))
        mean_loss_weighted_avg, err_loss_weighted_avg = np.mean(loss_list_weight_avg), np.std(loss_list_weight_avg)/np.sqrt(len(loss_list_weight_avg))
        mean_loss_weighted_avg_soft, err_loss_weighted_avg_soft = np.mean(loss_list_weight_avg_soft), np.std(loss_list_weight_avg_soft)/np.sqrt(len(loss_list_weight_avg_soft))
        mean_loss_equal_weights, err_loss_equal_weights = np.mean(loss_list_equal_weights), np.std(loss_list_equal_weights)/np.sqrt(len(loss_list_equal_weights))
        if sim_params['most_recent']:
                mean_loss_most_recent, err_loss_most_recent = np.mean(loss_list_most_recent), np.std(loss_list_most_recent)/np.sqrt(len(loss_list_most_recent))

        # percentage of improvement of QRA over the other models
        improvement_dayahead = (mean_loss_dayahead - mean_loss_QRA) / mean_loss_QRA * 100
        improvement_dayahead11h = (mean_loss_dayahead11h - mean_loss_QRA) / mean_loss_QRA * 100
        improvement_weekahead = (mean_loss_weekahead - mean_loss_QRA) / mean_loss_QRA * 100
        improvement_best_model = (mean_loss_best_model - mean_loss_QRA) / mean_loss_QRA * 100
        improvement_weighted_avg = (mean_loss_weighted_avg - mean_loss_QRA) / mean_loss_QRA * 100
        improvement_weighted_avg_soft = (mean_loss_weighted_avg_soft - mean_loss_QRA) / mean_loss_QRA * 100
        improvement_equal_weights = (mean_loss_equal_weights - mean_loss_QRA) / mean_loss_QRA * 100
        if sim_params['most_recent']:
                improvement_most_recent = (mean_loss_most_recent - mean_loss_QRA) / mean_loss_QRA * 100

        # plot seaborn barplot of mean rmse with std
        sns.set_theme(style="whitegrid")
        data = {'Forecaster': ['QRA', 'Best Selected', 'Equal Weights', 'Weighted Avg', 'Weighted Avg Soft', 'Day-ahead', 'Day-ahead 11h', 'Week-ahead'],
                'Mean': [mean_loss_QRA, mean_loss_best_model, mean_loss_equal_weights, mean_loss_weighted_avg, mean_loss_weighted_avg_soft, mean_loss_dayahead, mean_loss_dayahead11h, mean_loss_weekahead],
                'Std Error': [err_loss_QRA, err_loss_best_model, err_loss_equal_weights, err_loss_weighted_avg, err_loss_weighted_avg_soft, err_loss_dayahead, err_loss_dayahead11h, err_loss_weekahead],
                'Improvement (%)': [0, improvement_best_model, improvement_equal_weights, improvement_weighted_avg, improvement_weighted_avg_soft, improvement_dayahead, improvement_dayahead11h, improvement_weekahead]}
        if sim_params['most_recent']:
                data['Forecaster'].append('Most Recent')
                data['Mean'].append(mean_loss_most_recent)
                data['Std Error'].append(err_loss_most_recent)
                data['Improvement (%)'].append(improvement_most_recent)
        df = pd.DataFrame(data)
        
        if 'Coverage' in title:
                sns.barplot(x='Mean', y='Forecaster', data=df, alpha=0.5, color='green')
                # add error bars
                plt.errorbar(df['Mean'], df['Forecaster'], xerr=df['Std Error'], fmt='o')
                for i in range(len(df)):
                        plt.text(df['Mean'][i] + df['Std Error'][i], i, f" {df['Mean'][i]:.2f}", va='center', ha='left', rotation=0, fontsize=10)
                # plot vertical dashed red line for the maximum mean value
                plt.axvline(0.8, color='red', linestyle='dashed')
                plt.title(title)
        elif 'Sharpness' in title:
                sns.barplot(x='Mean', y='Forecaster', data=df, alpha=0.5, color='purple')
                # add error bars
                plt.errorbar(df['Mean'], df['Forecaster'], xerr=df['Std Error'], fmt='o')
                for i in range(len(df)):
                        plt.text(df['Mean'][i] + df['Std Error'][i], i, f" {df['Mean'][i]:.2f}", va='center', ha='left', rotation=0, fontsize=10)
                # plot vertical dashed red line for the maximum mean value
                plt.axvline(df['Mean'].min(), color='red', linestyle='dashed')
                plt.title(title)
        elif 'Coverage' or 'Sharpness' not in title:
                sns.barplot(x='Mean', y='Forecaster', data=df, alpha=0.5, color='blue')
                # add error bars
                plt.errorbar(df['Mean'], df['Forecaster'], xerr=df['Std Error'], fmt='o')
                # add percentage of improvement where text rotation is horizontal and smaller
                for i in range(len(df)):
                        plt.text(df['Mean'][i] + df['Std Error'][i], i, f" {df['Improvement (%)'][i]:.2f}%", va='center', ha='left', rotation=0, fontsize=10)
                # plot vertical dashed red line for the minimum mean value
                plt.axvline(df['Mean'].min(), color='red', linestyle='dashed')
                plt.title(f'{title} (QRA % improvement over other models)')
        else:
                # raise an error if the title is not correct
                raise ValueError('The title must contain either RMSE, Pinball Loss, Coverage or Sharpness')
                
        
        # save figure
        # split the title by space and join by underscore
        title = '_'.join(title.split())
        plt.savefig(f'{title}.png', dpi=300, bbox_inches='tight')
        plt.show()

In [ ]:
# rmse
list_QRA_over_dayahead = [1 if rmse_QRA <rmse_dayahead else 0 for rmse_QRA in rmse_list_QRA for rmse_dayahead in rmse_list_dayahead]
list_QRA_over_dayahead11h = [1 if rmse_QRA <rmse_dayahead11h else 0 for rmse_QRA in rmse_list_QRA for rmse_dayahead11h in rmse_list_dayahead11h]
list_QRA_over_weekahead = [1 if rmse_QRA <rmse_weekahead else 0 for rmse_QRA in rmse_list_QRA for rmse_weekahead in rmse_list_weekahead]
list_QRA_over_best = [1 if rmse_QRA <rmse_best else 0 for rmse_QRA in rmse_list_QRA for rmse_best in rmse_list_best]
list_QRA_over_equal_weights = [1 if rmse_QRA <rmse_equal_weights else 0 for rmse_QRA in rmse_list_QRA for rmse_equal_weights in rmse_list_equal_weights]
list_QRA_over_weight_avg = [1 if rmse_QRA <rmse_weight_avg else 0 for rmse_QRA in rmse_list_QRA for rmse_weight_avg in rmse_list_weight_avg]
list_QRA_over_weight_avg_soft = [1 if rmse_QRA <rmse_weight_avg_soft else 0 for rmse_QRA in rmse_list_QRA for rmse_weight_avg_soft in rmse_list_weight_avg_soft]
if sim_params['most_recent']:
    list_QRA_over_most_recent = [1 if rmse_QRA <rmse_most_recent else 0 for rmse_QRA in rmse_list_QRA for rmse_most_recent in rmse_list_most_recent]

# create dataframe
start_index = 0
end_index = -1
df_rmse = pd.DataFrame({'QRA': rmse_list_QRA, 'Day-ahead': rmse_list_dayahead, 'Day-ahead 11h': rmse_list_dayahead11h, 'Week-ahead': rmse_list_weekahead, 'Best Selected': rmse_list_best, 'Equal Weights': rmse_list_equal_weights, 'Weighted Avg': rmse_list_weight_avg, 'Weighted Avg Soft': rmse_list_weight_avg_soft})
if sim_params['most_recent']:
    df_rmse['Most Recent'] = rmse_list_most_recent
df_rmse = df_rmse.iloc[start_index:end_index]

# frequency QRA has the lowest RMSE over the other models df_rmse
n_best = df_rmse.apply(lambda x: x.idxmin(), axis=1).value_counts()/df_rmse.shape[0]
print('Frequency models with the lowest RMSE:')
print(n_best )
print(' ')
# frequency QRA has the highest RMSE over the other models df_rmse
n_worst = df_rmse.apply(lambda x: x.idxmax(), axis=1).value_counts()/df_rmse.shape[0]
print('Frequency models with the highest RMSE:')
print(n_worst)
print(' ')

# create dataframe where lowest RMSE is 1 and highest RMSE is -1 and others are 0 per date
df_binary = pd.DataFrame(index=df_rmse.index, columns=df_rmse.columns)
for date in df_rmse.index:
    df_binary.loc[date] = np.where(df_rmse.loc[date] == df_rmse.loc[date].min(), float(1.0), 
                        np.where(df_rmse.loc[date] == df_rmse.loc[date].max(), float(-1.0), 
                        float(0.0)))

# plot heatmap for df_binary
fig, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(df_binary.T.astype(float), cmap='RdYlGn', ax=ax)
plt.xlabel('date')
plt.ylabel('forecasters')
plt.title('Best RMSE per date')
plt.show()

# # create a dataframe where the value of RMSE from ensemble_Q50 and from week-ahead remain the same while the rest are 0
# df_binary_weekahead = pd.DataFrame(index=df_rmse.index, columns=df_rmse.columns)
# for date in df_rmse.index:
#     df_binary_weekahead.loc[date] = np.where(df_rmse.loc[date] == df_rmse.loc[date]['QRA'], df_rmse.loc[date]['QRA'], 
#                         #np.where(df_rmse.loc[date] == df_rmse.loc[date]['Week-ahead'], df_rmse.loc[date]['Week-ahead'], 
#                         float(0.0))
# df_binary_weekahead = df_binary_weekahead.astype(float)

# # plot heatmap for df_binary_weekahead
# fig, ax = plt.subplots(figsize=(20, 10))
# sns.heatmap(df_binary_weekahead.T, cmap='viridis', ax=ax)
# plt.xlabel('date')
# plt.ylabel('forecasters')
# plt.title('Best RMSE per date')
# plt.show()


In [ ]:
# pinball loss 0.1
list_QRA_over_dayahead_01 = [1 if pinball_loss_QRA <pinball_loss_dayahead else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_dayahead in pinball_loss_01_list_dayahead]
list_QRA_over_dayahead11h_01 = [1 if pinball_loss_QRA <pinball_loss_dayahead11h else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_dayahead11h in pinball_loss_01_list_dayahead11h]
list_QRA_over_weekahead_01 = [1 if pinball_loss_QRA <pinball_loss_weekahead else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_weekahead in pinball_loss_01_list_weekahead]
list_QRA_over_best_01 = [1 if pinball_loss_QRA <pinball_loss_best else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_best in pinball_loss_01_list_best]
list_QRA_over_equal_weights_01 = [1 if pinball_loss_QRA <pinball_loss_equal_weights else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_equal_weights in pinball_loss_01_list_equal_weights]
list_QRA_over_weight_avg_01 = [1 if pinball_loss_QRA <pinball_loss_weight_avg else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_weight_avg in pinball_loss_01_list_weight_avg]
list_QRA_over_weight_avg_soft_01 = [1 if pinball_loss_QRA <pinball_loss_weight_avg_soft else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_weight_avg_soft in pinball_loss_01_list_weight_avg_soft]
if sim_params['most_recent']:
    list_QRA_over_most_recent_01 = [1 if pinball_loss_QRA <pinball_loss_most_recent else 0 for pinball_loss_QRA in pinball_loss_01_list_QRA for pinball_loss_most_recent in pinball_loss_01_list_most_recent]

# create dataframe
start_index = 0
end_index = -1
df_pinball_q10 = pd.DataFrame({'QRA': pinball_loss_01_list_QRA, 'Day-ahead': pinball_loss_01_list_dayahead, 'Day-ahead 11h': pinball_loss_01_list_dayahead11h, 'Week-ahead': pinball_loss_01_list_weekahead, 'Best Selected': pinball_loss_01_list_best, 'Equal Weights': pinball_loss_01_list_equal_weights, 'Weighted Avg': pinball_loss_01_list_weight_avg, 'Weighted Avg Soft': pinball_loss_01_list_weight_avg_soft})
if sim_params['most_recent']:
    df_pinball_q10['Most Recent'] = pinball_loss_01_list_most_recent
df_pinball_q10 = df_pinball_q10.iloc[start_index:end_index]

# frequency QRA has the lowest Pinball over the other models df_pinball_q10
n_best = df_pinball_q10.apply(lambda x: x.idxmin(), axis=1).value_counts()/df_pinball_q10.shape[0]
print('Frequency models with the lowest Pinball Loss 0.1:')
print(n_best)
print('sum', n_best.sum())
print(' ')
# frequency QRA has the highest Pinball over the other models df_pinball_q10
n_worst = df_pinball_q10.apply(lambda x: x.idxmax(), axis=1).value_counts()/df_pinball_q10.shape[0]
print('Frequency models with the highest Pinball Loss 0.1:')
print(n_worst)
print('sum', n_worst.sum())
print(' ')

# create dataframe where lowest Pinball is 1 and highest Pinball is -1 and others are 0 per date
df_binary = pd.DataFrame(index=df_pinball_q10.index, columns=df_pinball_q10.columns)
for date in df_pinball_q10.index:
    df_binary.loc[date] = np.where(df_pinball_q10.loc[date] == df_pinball_q10.loc[date].min(), float(1.0), 
                        np.where(df_pinball_q10.loc[date] == df_pinball_q10.loc[date].max(), float(-1.0), 
                        float(0.0)))

# plot heatmap for df_binary
fig, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(df_binary.T.astype(float), cmap='RdYlGn', ax=ax)
plt.xlabel('date')
plt.ylabel('forecasters')
plt.title('Best Pinball Loss 0.1 per date')
plt.show()

In [ ]:
# pinball loss 0.9
list_QRA_over_dayahead_09 = [1 if pinball_loss_QRA <pinball_loss_dayahead else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_dayahead in pinball_loss_09_list_dayahead]
list_QRA_over_dayahead11h_09 = [1 if pinball_loss_QRA <pinball_loss_dayahead11h else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_dayahead11h in pinball_loss_09_list_dayahead11h]
list_QRA_over_weekahead_09 = [1 if pinball_loss_QRA <pinball_loss_weekahead else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_weekahead in pinball_loss_09_list_weekahead]
list_QRA_over_best_09 = [1 if pinball_loss_QRA <pinball_loss_best else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_best in pinball_loss_09_list_best]
list_QRA_over_equal_weights_09 = [1 if pinball_loss_QRA <pinball_loss_equal_weights else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_equal_weights in pinball_loss_09_list_equal_weights]
list_QRA_over_weight_avg_09 = [1 if pinball_loss_QRA <pinball_loss_weight_avg else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_weight_avg in pinball_loss_09_list_weight_avg]
list_QRA_over_weight_avg_soft_09 = [1 if pinball_loss_QRA <pinball_loss_weight_avg_soft else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_weight_avg_soft in pinball_loss_09_list_weight_avg_soft]
if sim_params['most_recent']:
    list_QRA_over_most_recent_09 = [1 if pinball_loss_QRA <pinball_loss_most_recent else 0 for pinball_loss_QRA in pinball_loss_09_list_QRA for pinball_loss_most_recent in pinball_loss_09_list_most_recent]
    
# create dataframe
start_index = 0
end_index = -1
df_pinball_q90 = pd.DataFrame({'QRA': pinball_loss_09_list_QRA, 'Day-ahead': pinball_loss_09_list_dayahead, 'Day-ahead 11h': pinball_loss_09_list_dayahead11h, 'Week-ahead': pinball_loss_09_list_weekahead, 'Best Selected': pinball_loss_09_list_best, 'Equal Weights': pinball_loss_09_list_equal_weights, 'Weighted Avg': pinball_loss_09_list_weight_avg, 'Weighted Avg Soft': pinball_loss_09_list_weight_avg_soft})
if sim_params['most_recent']:
    df_pinball_q90['Most Recent'] = pinball_loss_09_list_most_recent
df_pinball_q90 = df_pinball_q90.iloc[start_index:end_index]

# frequency QRA has the lowest Pinball over the other models df_pinball_q90
n_best = df_pinball_q90.apply(lambda x: x.idxmin(), axis=1).value_counts()/df_pinball_q90.shape[0]
print('Frequency models with the lowest Pinball Loss 0.9:')
print(n_best)
print('sum', n_best.sum())
print(' ')
# frequency QRA has the highest Pinball over the other models df_pinball_q90
n_worst = df_pinball_q90.apply(lambda x: x.idxmax(), axis=1).value_counts()/df_pinball_q90.shape[0]
print('Frequency models with the highest Pinball Loss 0.9:')
print(n_worst)
print('sum', n_worst.sum())

# create dataframe where lowest Pinball is 1 and highest Pinball is -1 and others are 0 per date
df_binary = pd.DataFrame(index=df_pinball_q90.index, columns=df_pinball_q90.columns)
for date in df_pinball_q90.index:
    df_binary.loc[date] = np.where(df_pinball_q90.loc[date] == df_pinball_q90.loc[date].min(), float(1.0), 
                        np.where(df_pinball_q90.loc[date] == df_pinball_q90.loc[date].max(), float(-1.0), 
                        float(0.0)))
start_index = 0
end_index = -1
df_binary = df_binary.iloc[start_index:end_index]
# plot heatmap for df_binary
fig, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(df_binary.T.astype(float), cmap='RdYlGn', ax=ax)
plt.xlabel('date')
plt.ylabel('forecasters')
plt.title('Best Pinball Loss 0.9 per date')
plt.show()

In [ ]:
# plot barplot
# rmse
title = 'RMSE by forecaster'
if sim_params['most_recent']:
    barplot(sim_params, title, rmse_list_QRA, rmse_list_dayahead, rmse_list_dayahead11h, rmse_list_weekahead, rmse_list_best, rmse_list_weight_avg, rmse_list_weight_avg_soft, rmse_list_equal_weights, rmse_list_most_recent)
else:
    barplot(sim_params, title, rmse_list_QRA, rmse_list_dayahead, rmse_list_dayahead11h, rmse_list_weekahead, rmse_list_best, rmse_list_weight_avg, rmse_list_weight_avg_soft, rmse_list_equal_weights)

# pinball loss 0.1
title = 'Pinball Loss 0.1 by forecaster'
if sim_params['most_recent']:
    barplot(sim_params, title, pinball_loss_01_list_QRA, pinball_loss_01_list_dayahead, pinball_loss_01_list_dayahead11h, pinball_loss_01_list_weekahead, pinball_loss_01_list_best, pinball_loss_01_list_weight_avg, pinball_loss_01_list_weight_avg_soft, pinball_loss_01_list_equal_weights, pinball_loss_01_list_most_recent)
else:
    barplot(sim_params, title, pinball_loss_01_list_QRA, pinball_loss_01_list_dayahead, pinball_loss_01_list_dayahead11h, pinball_loss_01_list_weekahead, pinball_loss_01_list_best, pinball_loss_01_list_weight_avg, pinball_loss_01_list_weight_avg_soft, pinball_loss_01_list_equal_weights)

# pinball loss 0.9
title = 'Pinball Loss 0.9 by forecaster'
if sim_params['most_recent']:
    barplot(sim_params, title, pinball_loss_09_list_QRA, pinball_loss_09_list_dayahead, pinball_loss_09_list_dayahead11h, pinball_loss_09_list_weekahead, pinball_loss_09_list_best, pinball_loss_09_list_weight_avg, pinball_loss_09_list_weight_avg_soft, pinball_loss_09_list_equal_weights, pinball_loss_09_list_most_recent)
else:
    barplot(sim_params, title, pinball_loss_09_list_QRA, pinball_loss_09_list_dayahead, pinball_loss_09_list_dayahead11h, pinball_loss_09_list_weekahead, pinball_loss_09_list_best, pinball_loss_09_list_weight_avg, pinball_loss_09_list_weight_avg_soft, pinball_loss_09_list_equal_weights)

# winkler score
title = 'Winkler Score by forecaster'
if sim_params['most_recent']:
    barplot(sim_params, title, winkler_score_list_QRA, winkler_score_list_dayahead, winkler_score_list_dayahead11h, winkler_score_list_weekahead, winkler_score_list_best, winkler_score_list_weight_avg, winkler_score_list_weight_avg_soft, winkler_score_list_equal_weights, winkler_score_list_most_recent)
else:
    barplot(sim_params, title, winkler_score_list_QRA, winkler_score_list_dayahead, winkler_score_list_dayahead11h, winkler_score_list_weekahead, winkler_score_list_best, winkler_score_list_weight_avg, winkler_score_list_weight_avg_soft, winkler_score_list_equal_weights)

# coverage
title = 'Coverage by forecaster'
if sim_params['most_recent']:
    barplot(sim_params, title, coverage_list_QRA, coverage_list_dayahead, coverage_list_dayahead11h, coverage_list_weekahead, coverage_list_best, coverage_list_weight_avg, coverage_list_weight_avg_soft, coverage_list_equal_weights, coverage_list_most_recent)
else:
    barplot(sim_params, title, coverage_list_QRA, coverage_list_dayahead, coverage_list_dayahead11h, coverage_list_weekahead, coverage_list_best, coverage_list_weight_avg, coverage_list_weight_avg_soft, coverage_list_equal_weights)

# sharpness
title = 'Sharpness by forecaster'
if sim_params['most_recent']:
    barplot(sim_params, title, sharpness_list_QRA, sharpness_list_dayahead, sharpness_list_dayahead11h, sharpness_list_weekahead, sharpness_list_best, sharpness_list_weight_avg, sharpness_list_weight_avg_soft, sharpness_list_equal_weights, sharpness_list_most_recent)
else:
    barplot(sim_params, title, sharpness_list_QRA, sharpness_list_dayahead, sharpness_list_dayahead11h, sharpness_list_weekahead, sharpness_list_best, sharpness_list_weight_avg, sharpness_list_weight_avg_soft, sharpness_list_equal_weights)

In [ ]:
print('Total number of days:', len(rmse_list_QRA))
print('Total number of years:', len(rmse_list_QRA_month_year)/12)
print(' ')

if sim_params['most_recent']:
    df_freq = pd.DataFrame({
    'model': ['dayahead', 'dayahead11h', 'weekahead', 'best_model', 'equal_weights', 'weight_avg', 'weight_avg_soft', 'most_recent'],
    'rmse': [sum(list_QRA_over_dayahead)/len(list_QRA_over_dayahead), sum(list_QRA_over_dayahead11h)/len(list_QRA_over_dayahead11h), sum(list_QRA_over_weekahead)/len(list_QRA_over_weekahead), sum(list_QRA_over_best)/len(list_QRA_over_best), sum(list_QRA_over_equal_weights)/len(list_QRA_over_equal_weights), sum(list_QRA_over_weight_avg)/len(list_QRA_over_weight_avg), sum(list_QRA_over_weight_avg_soft)/len(list_QRA_over_weight_avg_soft), sum(list_QRA_over_most_recent)/len(list_QRA_over_most_recent)],
    'pinball_loss_01': [sum(list_QRA_over_dayahead_01)/len(list_QRA_over_dayahead_01), sum(list_QRA_over_dayahead11h_01)/len(list_QRA_over_dayahead11h_01), sum(list_QRA_over_weekahead_01)/len(list_QRA_over_weekahead_01), sum(list_QRA_over_best_01)/len(list_QRA_over_best_01), sum(list_QRA_over_equal_weights_01)/len(list_QRA_over_equal_weights_01), sum(list_QRA_over_weight_avg_01)/len(list_QRA_over_weight_avg_01), sum(list_QRA_over_weight_avg_soft_01)/len(list_QRA_over_weight_avg_soft_01), sum(list_QRA_over_most_recent_01)/len(list_QRA_over_most_recent_01)],
    'pinball_loss_09': [sum(list_QRA_over_dayahead_09)/len(list_QRA_over_dayahead_09), sum(list_QRA_over_dayahead11h_09)/len(list_QRA_over_dayahead11h_09), sum(list_QRA_over_weekahead_09)/len(list_QRA_over_weekahead_09), sum(list_QRA_over_best_09)/len(list_QRA_over_best_09), sum(list_QRA_over_equal_weights_09)/len(list_QRA_over_equal_weights_09), sum(list_QRA_over_weight_avg_09)/len(list_QRA_over_weight_avg_09), sum(list_QRA_over_weight_avg_soft_09)/len(list_QRA_over_weight_avg_soft_09), sum(list_QRA_over_most_recent_09)/len(list_QRA_over_most_recent_09)]
})

else:
    # create dataframe with the frequency results with header 'QRA over model'
    df_freq = pd.DataFrame({
    'model': ['dayahead', 'dayahead11h', 'weekahead', 'best_model', 'equal_weights', 'weight_avg', 'weight_avg_soft'],
    'rmse': [sum(list_QRA_over_dayahead)/len(list_QRA_over_dayahead), sum(list_QRA_over_dayahead11h)/len(list_QRA_over_dayahead11h), sum(list_QRA_over_weekahead)/len(list_QRA_over_weekahead), sum(list_QRA_over_best)/len(list_QRA_over_best), sum(list_QRA_over_equal_weights)/len(list_QRA_over_equal_weights), sum(list_QRA_over_weight_avg)/len(list_QRA_over_weight_avg), sum(list_QRA_over_weight_avg_soft)/len(list_QRA_over_weight_avg_soft)],
    'pinball_loss_01': [sum(list_QRA_over_dayahead_01)/len(list_QRA_over_dayahead_01), sum(list_QRA_over_dayahead11h_01)/len(list_QRA_over_dayahead11h_01), sum(list_QRA_over_weekahead_01)/len(list_QRA_over_weekahead_01), sum(list_QRA_over_best_01)/len(list_QRA_over_best_01), sum(list_QRA_over_equal_weights_01)/len(list_QRA_over_equal_weights_01), sum(list_QRA_over_weight_avg_01)/len(list_QRA_over_weight_avg_01), sum(list_QRA_over_weight_avg_soft_01)/len(list_QRA_over_weight_avg_soft_01)],
    'pinball_loss_09': [sum(list_QRA_over_dayahead_09)/len(list_QRA_over_dayahead_09), sum(list_QRA_over_dayahead11h_09)/len(list_QRA_over_dayahead11h_09), sum(list_QRA_over_weekahead_09)/len(list_QRA_over_weekahead_09), sum(list_QRA_over_best_09)/len(list_QRA_over_best_09), sum(list_QRA_over_equal_weights_09)/len(list_QRA_over_equal_weights_09), sum(list_QRA_over_weight_avg_09)/len(list_QRA_over_weight_avg_09), sum(list_QRA_over_weight_avg_soft_09)/len(list_QRA_over_weight_avg_soft_09)]
})

# plot heatmap
import seaborn as sns
import matplotlib.pyplot as plt

def plot_heatmap(df, title):
    plt.figure(figsize=(10, 5))
    sns.heatmap(df.set_index('model').T, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
    plt.title(title)
    # save figure
    plt.savefig(f'{col}_frequency.png', dpi=300, bbox_inches='tight')
    plt.show()

for col in df_freq.columns:
    if col != 'model':
        plot_heatmap(df_freq[['model', col]], title=f'Frequency QRA outperforms other models for {col}')


In [ ]:
import scikit_posthocs as sp

def statistical_hypothesis_testing(sim_params, title, loss_list_QRA, loss_list_dayahead, loss_list_dayahead11h, loss_list_weekahead, loss_best_model, loss_weight_avg, loss_weight_avg_soft, loss_equal_weights, loss_most_recent=None):
    """ Perform statistical hypothesis testing using the Friedman test and posthoc Nemenyi test 
    """
    # Construct the dictionary from the input lists
    dict_data = {
        'QRA': loss_list_QRA,
        'dayahead': loss_list_dayahead,
        'dayahead11h': loss_list_dayahead11h,
        'weekahead': loss_list_weekahead,
        'selection': loss_best_model,
        'weight_avg': loss_weight_avg,
        'weight_avg_soft': loss_weight_avg_soft,
        'weight_equal': loss_equal_weights
    }
    if sim_params['most_recent']:
        dict_data['most_recent'] = loss_most_recent

    # Transform the dictionary into a DataFrame
    data = (
        pd.DataFrame(dict_data)
        .rename_axis('days')  # Set the index name to 'days'
        .melt(                # Melt the DataFrame to long format
            var_name='model',
            value_name='loss',
            ignore_index=False,
        )
        .reset_index()        # Reset the index to include 'days' as a column
    )
    # Perform posthoc Nemenyi Friedman test
    tests_results = sp.posthoc_nemenyi_friedman(data, y_col='loss', group_col='model', block_col='days',  block_id_col='days', melted=True) #
    # Define the colormap and heatmap arguments
    cmap = ['1', '#fb6a4a',  '#08306b',  '#4292c6', '#c6dbef']
    heatmap_args = {
        'cmap': cmap,
        'linewidths': 0.25,
        'linecolor': '0.5',
        'clip_on': False,
        'square': True,
        'cbar_ax_bbox': [0.80, 0.35, 0.04, 0.3]
    }
    
    # Compute the average rank of each model
    print('------------------------------------')
    avg_rank = data.groupby('days').loss.rank(method='average').groupby(data.model).mean()
    print(f"Average rank of the models - {avg_rank}")

    # Plot the heatmap
    plt.title(f"P-values posthoc Nemenyi test for {title}")
    # mask the upper triangle
    mask = np.triu(np.ones_like(tests_results, dtype=bool))
    tests_results_masked = tests_results.mask(mask)
    # print the p-values
    # add to column names the avg rank in square brackets
    # in bold if QRA
    tests_results_masked.columns = [f"{col} - {avg_rank[col]:.3f}" for col in tests_results_masked.columns]
    tests_results_masked.columns = [f"**{col}**" if 'QRA' in col else col for col in tests_results_masked.columns]
    tests_results_masked.index = [f"{col} - {avg_rank[col]:.3f}" for col in tests_results_masked.index]
    tests_results_masked.index = [f"**{col}**" if 'QRA' in col else col for col in tests_results_masked.index]
    # add header
    tests_results_masked.columns.name = 'Average Rank of the models'
    tests_results_masked.index.name = 'Average Rank of the models'

    # figure size
    sp.sign_plot(tests_results_masked, **heatmap_args)
    # save image
    # split the title
    title = title.replace(' ', '_')
    plt.savefig(f'p_values_posthoc_nemenyi_test_{title}.png', dpi=300, bbox_inches='tight')
    plt.show()


    


In [ ]:
# rmse
if sim_params['most_recent']:
    title = 'RMSE'
    statistical_hypothesis_testing(sim_params, title, rmse_list_QRA, rmse_list_dayahead, rmse_list_dayahead11h, rmse_list_weekahead, rmse_list_best, rmse_list_weight_avg, rmse_list_weight_avg_soft, rmse_list_equal_weights, rmse_list_most_recent)
else:
    title = 'RMSE'
    statistical_hypothesis_testing(sim_params, title, rmse_list_QRA, rmse_list_dayahead, rmse_list_dayahead11h, rmse_list_weekahead, rmse_list_best, rmse_list_weight_avg, rmse_list_weight_avg_soft, rmse_list_equal_weights)

# pinball loss 0.1
if sim_params['most_recent']:
    title = 'Pinball Loss 0.1'
    statistical_hypothesis_testing(sim_params, title, pinball_loss_01_list_QRA, pinball_loss_01_list_dayahead, pinball_loss_01_list_dayahead11h, pinball_loss_01_list_weekahead, pinball_loss_01_list_best, pinball_loss_01_list_weight_avg, pinball_loss_01_list_weight_avg_soft, pinball_loss_01_list_equal_weights, pinball_loss_01_list_most_recent)
else:
    title = 'Pinball Loss 0.1'
    statistical_hypothesis_testing(sim_params, title, pinball_loss_01_list_QRA, pinball_loss_01_list_dayahead, pinball_loss_01_list_dayahead11h, pinball_loss_01_list_weekahead, pinball_loss_01_list_best, pinball_loss_01_list_weight_avg, pinball_loss_01_list_weight_avg_soft, pinball_loss_01_list_equal_weights)

# pinball loss 0.9
if sim_params['most_recent']:
    title = 'Pinball Loss 0.9'
    statistical_hypothesis_testing(sim_params, title, pinball_loss_09_list_QRA, pinball_loss_09_list_dayahead, pinball_loss_09_list_dayahead11h, pinball_loss_09_list_weekahead, pinball_loss_09_list_best, pinball_loss_09_list_weight_avg, pinball_loss_09_list_weight_avg_soft, pinball_loss_09_list_equal_weights, pinball_loss_09_list_most_recent)
else:
    title = 'Pinball Loss 0.9'
    statistical_hypothesis_testing(sim_params, title, pinball_loss_09_list_QRA, pinball_loss_09_list_dayahead, pinball_loss_09_list_dayahead11h, pinball_loss_09_list_weekahead, pinball_loss_09_list_best, pinball_loss_09_list_weight_avg, pinball_loss_09_list_weight_avg_soft, pinball_loss_09_list_equal_weights)
